In [1]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [23]:
pip install seaborn

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install scikit-learn


Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
df_find = pd.read_csv(r"C:\Users\avina\Downloads\spam_ham_dataset.csv")

df_find.head()

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [35]:
print(df_find.dtypes)


label           object
text            object
label_num        int64
text_cleaned    object
dtype: object


In [4]:
df_find.rename(columns={'Unnamed: 0':'reg'},inplace=True)
df_find.head(5)

,reg,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [5]:
df_find.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5171 entries, 0 to 5170
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   reg        5171 non-null   int64 
 1   label      5171 non-null   object
 2   text       5171 non-null   object
 3   label_num  5171 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 161.7+ KB


In [6]:
df_find.drop(columns=['reg'],inplace=True)

df_find.head(5)

,label,text,label_num
0,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,spam,"Subject: photoshop , windows , office . cheap ...",1
4,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [7]:
df_find['label_num'] = df_find['label'].map({'ham' : 0,'spam' : 1})
df_find.head()

,label,text,label_num
0,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,spam,"Subject: photoshop , windows , office . cheap ...",1
4,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [8]:
import nltk
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.tree import DecisionTreeClassifier
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\avina\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\avina\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [9]:
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')  # Ensure wordnet is downloaded for lemmatization

lemmatizer = WordNetLemmatizer()

stop_words =  set(stopwords.words('english'))

def text_cleaner(text):
    text = re.sub(r'[^0-9a-zA-Z\s]', '', text)  # Remove punctuation
    words = word_tokenize(text.lower())  # Tokenization and lowercase
    words = [lemmatizer.lemmatize(w) for w in words if w not in stop_words]  # Lemmatization
    return ' '.join(words)

df_find['text_cleaned'] = df_find['text'].apply(text_cleaner)


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\avina\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [10]:
df_find['text_cleaned'] = df_find['text'].apply(text_cleaner)

In [11]:
print(df_find['text_cleaned'])

0       subject enron methanol meter 988291 follow not...
1       subject hpl nom january 9 2001 see attached fi...
2       subject neon retreat ho ho ho around wonderful...
3       subject photoshop window office cheap main tre...
4       subject indian spring deal book teco pvr reven...
                              ...                        
5166    subject put 10 ft transport volume decreased 2...
5167    subject 3 4 2000 following noms hpl take extra...
5168    subject calpine daily gas nomination julie men...
5169    subject industrial worksheet august 2000 activ...
5170    subject important online banking alert dear va...
Name: text_cleaned, Length: 5171, dtype: object


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

tfidf = TfidfVectorizer(max_features =5000)
X = tfidf.fit_transform(df_find['text_cleaned']).toarray()
Y = df_find['label']

In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

LG = LogisticRegression()
LG.fit(X_train,Y_train)
predict_LG = LG.predict(X_test)

LG.score(X_test,Y_test)


0.9845410628019323

In [14]:
print(classification_report(Y_test,predict_LG))

              precision    recall  f1-score   support

         ham       0.99      0.99      0.99       742
        spam       0.97      0.98      0.97       293

    accuracy                           0.98      1035
   macro avg       0.98      0.98      0.98      1035
weighted avg       0.98      0.98      0.98      1035



In [15]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier(max_depth=10,min_samples_split=5)
DT.fit(X_train,Y_train)
predict_DT = DT.predict(X_test)

DT.score(X_test,Y_test)



0.9130434782608695

In [16]:
print(classification_report(predict_DT,Y_test))

              precision    recall  f1-score   support

         ham       0.89      0.99      0.94       672
        spam       0.97      0.78      0.86       363

    accuracy                           0.91      1035
   macro avg       0.93      0.88      0.90      1035
weighted avg       0.92      0.91      0.91      1035



In [17]:
# from sklearn.ensemble import GradientBoostingClassifier

# GB = GradientBoostingClassifier()
# GB.fit(X_train,Y_train)
# predict_GB = GB.predict(X_test)

In [18]:
# GB.score(X_test,Y_test)

In [19]:
# print(classification_report(predict_GB,Y_test))

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer

def output(n):
    if n == 1:
        return 'spam'
    elif n == 0:
        return 'not spam'
    else:
        return 'not a spam and nor ham'



def testing(email):
    texting = {'text':[email]}
    new_dataFrame = pd.DataFrame(texting)
    new_dataFrame['text'] = new_dataFrame['text'].apply(text_cleaner)
    new_x = new_dataFrame['text']
    new_xv = tfidf.transform(new_x)
    LG_predict = LG.predict(new_xv)
    DT_predict = DT.predict(new_xv)

    return f"LG_prediction {LG_predict[0]}, DT_prediction {DT_predict[0]}"


In [21]:
email = str(input())
testing(email)

 Subject: Meeting Reminder: Project Update Body:  Hi [Recipient's Name],  Just a quick reminder about our project update meeting tomorrow at 10:00 AM in the conference room. We will discuss the following topics:  Current progress on the ongoing tasks Upcoming deadlines and responsibilities Addressing any blockers or challenges Please be prepared to give a brief update on your assigned tasks. If you have any additional items you'd like to add to the agenda, feel free to let me know.  Looking forward to catching up tomorrow.  Best regards, [Your Name] [Your Position] [Your Company Name]


'LG_prediction ham, DT_prediction spam'